In [1]:
pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 410 kB 5.0 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import nlpaug.augmenter.char as nac
from os import listdir
from os.path import isfile, join
import re
import string
import unicodedata

In [4]:
input_path='/content/drive/MyDrive/Data/CNB/text_ocr'
output_path='/content/drive/MyDrive/Data/MyFilesCNB'

In [8]:
punctuations=string.punctuation+'¡¿“”-«»—£'
punctuations

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~¡¿“”-«»—£'

Se utilizara la libreria nlpaug, en particular OcrAug para simular error OCR.

## Niveles de ruido

Parametros por deafult de OcrAug:

    class nlpaug.augmenter.char.ocr.OcrAug(name='OCR_Aug', aug_char_min=2, aug_char_max=10, aug_char_p=0.3, aug_word_p=0.3, aug_word_min=1, aug_word_max=10, stopwords=None, tokenizer=None, reverse_tokenizer=None, verbose=0, stopwords_regex=None, min_char=1, dict_of_path=None)

Definimos tres niveles de ruido OCR usando los parametros que se muestran arriba.


| Ruido  | min_char | max_char | min_words | max_words |
|--------|----------|----------|-----------|-----------|
| low    | 1        | 2        | 1         | 2         |
| medium | 1        | 5        | 1         | 3         |
| high   | 2        | 10       | 1         | 10        |




Creamos las tres instancias corresponientes al ruido.

In [9]:
aug_low=nac.OcrAug(aug_char_min=1,aug_char_max=2,aug_word_min=1,aug_word_max=2)
aug_medium=nac.OcrAug(aug_char_min=1,aug_char_max=5,aug_word_min=1,aug_word_max=3)
aug_high=nac.OcrAug()
augmenters={'low':aug_low,'medium':aug_medium,'high':aug_high}

## Vocabulario

Formamos un vocabulario sobre el cual se entrenará usando todas las tesis a excepción de la tesis *Operación 23 de Septiembre*. La cual se usará para evaluar el desempeño del modelo que se entrene. Se usan los documentos que corresponden a tesis por la buena calidad de traducción OCR que se observa a simple vista.


### Una sola tesis

Obtenemos todos los documentos correspondientes a la tesis

In [10]:
filename='TESIS.LIC.reportaje.TAMARIZ Estrada, María C. Operación 23 de septiembre.'
path_tesis=join(input_path,filename)
files_tesis=[f for f in listdir(path_tesis) if isfile(join(path_tesis,f))]

Ordenamos los archivos pues se usara para evaluar correctores donde el contexto es importante.

In [11]:
files_tesis.sort(key=lambda f: int(re.sub('\D', '', f)))

Quitamos signos de puntuación.

In [12]:
translator = str.maketrans('','',punctuations)

Concatenamos todos el texto correspondiente a la tesis y se guardan en un archivo

In [28]:
with open(join(output_path,filename+'.txt'),'w') as outfile:
  for fname in files_tesis:
    with open(join(path_tesis,fname)) as infile:
      for i,line in enumerate(infile):
        temp=line.translate(translator).replace('"','').replace('\x0C', '')
        if temp!='\n':
          outfile.write(temp)

Cargamos el archivo.

In [29]:
lines=[]
with open(join(output_path,filename+'.txt'),'r') as f:
  lines=f.readlines()

In [31]:
lines[109:113]

['El 27 de noviembre de 2001 Vicente Fox ya como presidente recibe en el\n',
 'Archivo General de la Nación AGN los resultados de la investigación de la CNDH y\n',
 'responde con el Acuerdo por el que se disponen medidas para la procuración de justicia\n',
 'pagina1\n']

Se utilizaran los tres niveles de error OCR definidos anteriormente.

In [32]:
lines_aug={}
for key in augmenters:
  lines_aug[key]=augmenters[key].augment(lines)

In [33]:
init,end=109,113
for key in lines_aug:
  print('Nivel ruido OCR:{}\n{}'.format(key,lines_aug[key][init:end]))

Nivel ruido OCR:low
['El 27 de noviembre de 2001 Vicente Fox ya como presidente recibe en el', 'Archivo Cenera1 de la Nación AGN los resultados de la inve8ti9ación de la CNDH y', 'responde con e1 Acuerdo por el que se disponen medida8 para la procuración de justicia', 'pa9ina7']
Nivel ruido OCR:medium
['Bl 21 de noviembre de 2001 Vicente F0x ya como presidente recibe en el', 'Archivo General de la Nación ACN los resoltadu8 de la investigación de la GN0H y', 'responde con el Acuerdo por e1 que se disponen medidas paka la procuración de justicia', 'pa9inal']
Nivel ruido OCR:high
['El 27 de noviembre de 2001 Vicente Fox ya c0m0 presidente keci6e en e1', 'Archivo General de la Nación AGN l08 resultados de la inve8ti9ación de 1a GNOH y', 'responde con e1 Acuerdo por e1 que 8e disponen medidas para la procuración de jo8ticia', 'pa9ina7']


Guardamos los tres textos con los diferentes niveles de ruido OCR.

In [34]:
for level in lines_aug:
  with open(join(output_path,filename+'_'+level+'.txt'),'w') as outfile:
    for line in lines_aug[level]:
      outfile.write(line+'\n')

### Todas las tesis

Ahora formamos un documento mas grande concatenando todas las tesis restantes.

In [35]:
output_filename='all_tesis'
all_files=[f for f in listdir(input_path)]
files_tesis=[]
for f in all_files:
  if 'TESIS' in f:
    files_tesis.append(unicodedata.normalize('NFC',f))

files_tesis.remove(unicodedata.normalize('NFC',filename)) #keep one file for testing

with open(join(output_path,output_filename+'.txt'),'w') as outfile: 
  for filename in files_tesis:
    path_tesis=join(input_path,filename)
    pages=[f for f in listdir(path_tesis) if isfile(join(path_tesis,f))]
    pages.sort(key=lambda f: int(re.sub('\D', '', f)))
    for page_file in pages:
      with open(join(path_tesis,page_file)) as infile:
        for line in infile:
          temp=line.translate(translator).replace('"','')
          if temp!='\n':
            outfile.write(temp)

Leemos el archivo

In [36]:
with open(join(output_path,output_filename+'.txt'),'r') as f:
  all_tesis=f.readlines()

In [37]:
all_tesis[1010:1020]

['armado Durante 1964 destruyeron e incendiaron un puente de la familia\n',
 'Ibarra también dinamitaron el rancho Los cuatro compadres asaltaron\n',
 'y destruyeron una planta de luz Este grupo y en particular el año de\n',
 '1964 es considerado por López Limón como el inició de la lucha armada\n',
 'revolucionaria socialista apoyada en la doctrina marxista  \n',
 'Consideraron de utilidad la teoría guevarista del foco es decir\n',
 'consideraron que la rebelión creaba las condiciones para la lucha\n',
 'armada En lo que quedaba de 1964 y durante 1965 buscaron aumentar\n',
 'su capacidad de fuego y consolidar sus redes de apoyo entre los\n',
 'campesinos y para el 23 de septiembre 1965intentaron realizar el asalto\n']

Se utilizaran los tres niveles de error OCR definidos anteriormente.

In [38]:
all_tesis_aug={}
for key in augmenters:
  all_tesis_aug[key]=augmenters[key].augment(all_tesis)

In [39]:
init,end=1010,1020
for key in all_tesis_aug:
  print('Nivel ruido OCR:{}\n{}'.format(key,all_tesis_aug[key][init:end]))

Nivel ruido OCR:low
['armado Durante 1964 de8troyeron e incendiaron un puente de 1a familia', 'Ibarra tam6ién dinamitaron el kanchu Los cuatro compadres asaltaron', 'y destruyeron una planta de luz Este 9ropo y en particu1ak el año de', '1964 es considerado por López Limón como e1 inició de la lucha akmada', 'revolucionaria socialista apuyada en la ductkina marxista', 'C0n8ideraron de utilidad la teoría guevarista del foco es decir', 'consideraron que la rebelión creaba la8 condiciones paka la lucha', 'akmada En lo que quedaba de 1964 y dokante 1965 buscaron aumentar', 'su capacidad de fuego y consolidar sus redes de apoyo entke l0s', 'campesinos y paka el 23 de 8eptiem6re 1965intentaron realizar el asalto']
Nivel ruido OCR:medium
['akmad0 Durante 1964 destruyeron e incendiaron un poente de 1a familia', 'Ibarra también dinamitakun el kanch0 Lo8 cuatro compadres asaltaron', 'y destruyeron ona planta de luz Este grupo y en paktico1ar el año de', '1964 es considerado pok López Limón cum0 

Guardamos los tres textos con los diferentes niveles de ruido OCR.

In [40]:
for level in all_tesis_aug:
  with open(join(output_path,output_filename+'_'+level+'.txt'),'w') as outfile:
    for line in all_tesis_aug[level]:
      outfile.write(line+'\n')